In [1]:
import pandas as pd 
import glob
ROOTPATH = '/Users/zhenggong/Documents/Github/IBES_selected/capitaliq' # for importing and reference management 
import sys
sys.path.append(ROOTPATH)
import pandas as pd 
from cfg import SERVER_TIMEZONE,DBINFO
import os
import json
import psycopg2
from datetime import datetime, timedelta


def read_sql_to_df(sql, db, cursor):
    """Internal: execute sql and get dataframe as a return 
    
    Args:
        sql (str): sql to be executed 
        db (psycopg2.connect): connect to ciq target database 
        cursor (database connection.cursor): 
    
    Returns:
        pd.DataFrame: contains result for executed sql 
    """
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(data,columns = columns)
    except (Exception, psycopg2.DatabaseError) as error:
        db.rollback()
        print("Error: %s" % error)


def get_connection(dbInfo):
    """get the db connection using info in dbInfo json file
    
    Args:
        dbInfo (.json): user-pw for ciq target database 
    
    Returns:
        psycopg2.connect
    """
    with open(dbInfo,'r') as f:
        u = json.load(f)

    db = psycopg2.connect(
        host=u['host'],
        database=u["database"],
        user=u["user"],
        password=u["pwd"])
    return db

###################################################################################
def get_transcriptperson(ids, connection = None):


    sql = f"""
    SELECT ciqtranscriptperson.proid
    ,ciqtranscriptperson.transcriptpersonid
    ,ciqtranscriptperson.transcriptpersonname
    ,ciqtranscriptperson.companyname
    ,ciqprofessional.personid
    ,ciqprofessional.companyid
    ,ciqperson.firstname
    ,ciqperson.lastname
    ,ciqestimatebroker.brokername
    ,ciqestimatebroker.estimatebrokerid
    from ciqtranscriptperson 
    left join ciqprofessional on ciqtranscriptperson.proid = ciqprofessional.proid
    left join ciqperson on ciqprofessional.personid = ciqperson.personid
    left join ciqestimatebroker on ciqprofessional.companyid = ciqestimatebroker.companyid
    left join ciqcompany on ciqprofessional.companyid = ciqcompany.companyid
    where transcriptpersonid in ({', '.join([str(id) for id in ids])});
    """
    # print(sql)
    if connection is None:
        connection = get_connection(DBINFO)
    cursor = connection.cursor()

    return read_sql_to_df(sql, connection, cursor)



In [58]:
transcripts_paths = glob.glob('/Users/zhenggong/Documents/GitHub/transcriptPipeline/data/raw/full_transcript/*.parquet')[0:1000]

res = []
for transcript_path in transcripts_paths:
    sample_transcript = pd.read_parquet(transcript_path)
    sample_transcript = sample_transcript.query('transcriptcomponenttypeid == 3')[['transcriptpersonid', 'transcriptpersonname']].drop_duplicates(subset='transcriptpersonid')
    sample_transcript['tid'] = int(transcript_path.replace('.parquet', '').split('/')[-1].split('_')[0])
    res.append(sample_transcript)
sample_transcript = pd.concat(res)
print(len(sample_transcript))

sample_transcript = pd.merge(sample_transcript, pd.read_csv('/Users/zhenggong/Documents/GitHub/ba_thesis/data/processed/universeAugmented.csv').rename(columns={"transcriptid":"tid"})[['tid', 'calendaryear']],
                             left_on='tid', right_on='tid')
print(sample_transcript)

# get year range of the analysts
year_range = sample_transcript.groupby('transcriptpersonid')['calendaryear'].agg(['min', 'max']).reset_index()
year_range.columns = ['transcriptpersonid', 'min_year', 'max_year']

# catch some specific case
substring_to_check = 'Unknown'
sample_transcript = sample_transcript.query('~transcriptpersonname.str.contains(@substring_to_check)')
sample_transcript.drop_duplicates(subset='transcriptpersonid', inplace=True)
print(len(sample_transcript))

people_details = get_transcriptperson(sample_transcript['transcriptpersonid'].tolist())
people_details = people_details.dropna(subset=['proid', 'companyname'], how='all')
people_details = pd.merge(people_details, year_range, left_on='transcriptpersonid', right_on='transcriptpersonid')
people_details.sort_values('transcriptpersonname')

10251
       transcriptpersonid transcriptpersonname      tid  calendaryear
0                  219519              Kai Pan  1657164          2019
1                  263676      Elyse Greenspan  1657164          2019
2                  354649     Andrew Kligerman  1657164          2019
3                  260989        Meyer Shields  1657164          2019
4                  135333       Joshua Shanker  1657164          2019
...                   ...                  ...      ...           ...
10150              178511        Jason English  1262492          2017
10151              286675      Steven Strycula  1262492          2017
10152              102530      Kenneth Goldman  1262492          2017
10153              104293        Robert Moskow  1262492          2017
10154               97850         Andrew Lazar  1262492          2017

[10155 rows x 4 columns]
3109


,proid,transcriptpersonid,transcriptpersonname,companyname,personid,companyid,firstname,lastname,brokername,estimatebrokerid,min_year,max_year
1292,141259562.0,213539,A. Mark Finkelstein,None,26897041.0,100489358.0,A.,Finkelstein,Evercore ISI,2554.0,2013,2013
48,54044709.0,92376,Aaron Deer,None,54044708.0,25218138.0,Aaron,Deer,Sandler O'Neill,119.0,2018,2019
2271,273852659.0,321626,Aaron Grey,None,273852658.0,26092634.0,Aaron,Grey,TD Cowen,132.0,2016,2016
1371,141064775.0,223505,Aaron Kessler,None,37851557.0,40374081.0,Aaron,Kessler,Raymond James & Associates,114.0,2014,2014
441,54293786.0,103517,Aaron Rakers,None,38447837.0,9532654.0,Aaron,Rakers,Stifel,124.0,2014,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
2169,276180732.0,311148,Zachary Rosenberg,None,276180731.0,9914241.0,Zachary,Rosenberg,Baird,186.0,2017,2017
2556,410397038.0,356783,Zachary Silverberg,None,410397037.0,108000140.0,Zachary,Silverberg,Mizuho Securities USA LLC,2557.0,2020,2020
1492,181991178.0,237163,Zachary Sopcak,None,181991177.0,25232855.0,Zachary,Sopcak,Morgan Stanley,11.0,2018,2018
2887,649968002.0,459505,Zachary Weiner,None,649968001.0,22401937.0,Zachary,Weiner,Jefferies LLC,90.0,2021,2022


In [80]:
people_details.query("personid == 41074")

,proid,transcriptpersonid,transcriptpersonname,companyname,personid,companyid,firstname,lastname,brokername,estimatebrokerid,min_year,max_year


In [43]:
# recommendataions (buy or sell or HOLD)

# Focus on IRECCD (which is reconciled at IBES level) 
# etext and ereccd are explicit recommendation (can be less comparable)
recommend = pd.read_csv('/Users/zhenggong/Documents/GitHub/IBES_selected/IBES_csv/nahghdromygr63ao.csv', index_col=[0]).dropna(subset=['ANALYST']).reset_index(drop=True)
# restrict to only brokerage firms that comments are USFIRM
recommend = recommend.query('USFIRM == 1.0').sort_values(by=['ACTDATS'])
# filter out old data
recommend = recommend.query("ANNDATS > '2000-01-01'")

recommend['LastName'] = recommend['ANALYST'].str.split(' ').str[0]
recommend['FirstName'] = recommend['ANALYST'].str[-1]
# length of the data
print('The length of the data is: ' + str(len(recommend)))



/var/folders/0j/2k9yrkz575s8zxhdwtq0yrvm0000gn/T/ipykernel_95099/1916839552.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  recommend = pd.read_csv('/Users/zhenggong/Documents/GitHub/IBES_selected/IBES_csv/nahghdromygr63ao.csv', index_col=[0]).dropna(subset=['ANALYST']).reset_index(drop=True)


The length of the data is: 640087


In [44]:
# load in the broker table mapping 
broker_mapping = pd.read_csv('/Users/zhenggong/Documents/GitHub/IBES_selected/runnables/broker_match_v2_editted.csv', index_col=[0])
broker_mapping['ESTIMID'] = broker_mapping['broker_matching_name']

# groupby the recommendation to de-duplicate
recommend['count'] = 0
IBES_analyst = recommend[['ANALYST', 'ESTIMID', 'EMASKCD', 'count']].groupby(['ANALYST', 'EMASKCD', 'ESTIMID']).count().reset_index()
IBES_analyst = pd.merge(IBES_analyst, broker_mapping[['ESTIMID', 'choice']], left_on='ESTIMID', right_on='ESTIMID', how='left')
IBES_analyst['LastName'] = IBES_analyst['ANALYST'].str.split(' ').str[0]
IBES_analyst['FirstName'] = IBES_analyst['ANALYST'].str[-1]
# print(IBES_analyst)

recommend_augmented = pd.merge(recommend, broker_mapping[['ESTIMID', 'choice']], left_on='ESTIMID', right_on='ESTIMID', how='left').drop(columns=['count'])
recommend_augmented = pd.merge(recommend_augmented, IBES_analyst[['ANALYST', 'EMASKCD', 'ESTIMID', 'count']], 
                               left_on=['ANALYST', 'EMASKCD', 'ESTIMID'], 
                               right_on=['ANALYST', 'EMASKCD', 'ESTIMID'], how='left')
recommend_augmented['year'] = recommend_augmented['ACTDATS'].str.split('-').str[0].astype(int)
recommend_augmented

,TICKER,CUSIP,CNAME,OFTIC,ACTDATS,ESTIMID,ANALYST,ERECCD,ETEXT,IRECCD,...,ACTTIMS,REVDATS,REVTIMS,ANNDATS,ANNTIMS,LastName,FirstName,choice,count,year
0,BBC,08373910,BERGEN BRUNSWIG,BBC,2000-01-03,MONTSEC,YAFFE L,NaN,NEUTRAL,3.0,...,1960-01-01 13:48:34.000,2001-07-30,1960-01-01 16:04:16.000,2000-01-03,1960-01-01 13:48:34.000,YAFFE,L,2.0,31,2000
1,HPOL,41454910,HARRIS INTRACTVE,HPOL,2000-01-03,PIPER,MANDERFELD B,1,STRONG BUY,1.0,...,1960-01-01 14:26:24.000,2000-10-26,1960-01-01 17:18:13.000,2000-01-03,1960-01-01 14:26:24.000,MANDERFELD,B,112.0,125,2000
2,PRHC,74397710,PROVINCE HEALTH,PRHC,2000-01-03,DLJ,HINDELONG J,2,BUY,2.0,...,1960-01-01 16:26:38.000,2000-10-06,1960-01-01 16:23:31.000,2000-01-03,1960-01-01 16:26:38.000,HINDELONG,J,611.0,9,2000
3,DMRC,25380710,DIGIMARC CORP,DMRC,2000-01-03,PIPER,MUNSTER G,1,STRONG BUY,1.0,...,1960-01-01 12:44:26.000,2000-07-19,1960-01-01 16:37:28.000,2000-01-03,1960-01-01 12:44:26.000,MUNSTER,G,112.0,219,2000
4,IPC,45231710,ILLINOVA CORP,ILN,2000-01-03,CHICAGO,FORD D,1,BUY,1.0,...,1960-01-01 16:27:15.000,2000-02-01,1960-01-01 17:39:44.000,2000-01-03,1960-01-01 16:27:15.000,FORD,D,159.0,78,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640082,EMN,27743210,EASTMAN CHEMICAL,EMN,2024-02-15,REDBURN,ISAAC C,1,BUY,2.0,...,1960-01-01 05:25:35.000,2024-02-15,1960-01-01 05:25:35.000,2024-02-15,1960-01-01 05:19:00.000,ISAAC,C,1116.0,10,2024
640083,CLF,18589910,CLEVELAND-CLIFFS,CLF,2024-02-15,MORGAN,DE ALBA C,6,EQUALWT/IN-LINE,3.0,...,1960-01-01 00:22:41.000,2024-02-15,1960-01-01 00:22:41.000,2024-02-15,1960-01-01 00:16:00.000,DE,C,11.0,105,2024
640084,CPOW,45718710,INGREDION,INGR,2024-02-15,GOLDMAN,SAMUELSON A,1,BUY,2.0,...,1960-01-01 00:46:48.000,2024-02-15,1960-01-01 00:46:48.000,2024-02-15,1960-01-01 00:35:00.000,SAMUELSON,A,7.0,127,2024
640085,GRVI,39959A10,UPEXI,UPXI,2024-02-15,EURPACAP,GREY A,3,NEUTRAL,3.0,...,1960-01-01 08:24:18.000,2024-02-15,1960-01-01 08:24:18.000,2024-02-15,1960-01-01 07:55:00.000,GREY,A,2282.0,32,2024


In [45]:
# intermezzo 
# prepare for the next step

old_match = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Broker codes collection v2.xlsx', 'rb'),
                            sheet_name='Old match table').dropna(subset=['Baname'])
old_match.rename(columns={'Baname':'brokername_old', 'Baid': 'ESTIMID'}, inplace=True)
old_match

europe_match = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Broker codes collection v2.xlsx', 'rb'),
            sheet_name='Europe').dropna(subset=['Group name'])
europe_match.rename(columns={'Group name':'brokername_eu', 'Baid': 'ESTIMID'}, inplace=True)
europe_match

broker_transition_table = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Broker codes collection v2.xlsx', 'rb'),
            sheet_name='Top group table')
broker_transition_table.columns = ['sub', 'top', 'date1', 'date2']
broker_transition_table.drop(columns=['date1','date2'], inplace=True)
broker_transition_table


,sub,top
0,UBS,UBS
1,SBC WARBURG,SBC WARBURG
2,SBC WARBURG,UBS
3,DILLON READ,DILLON READ
4,DILLON READ,SBC WARBURG
...,...,...
499,WOOD & CO. FINANCIAL SERVICES,WOOD & CO. FINANCIAL SERVICES
500,WR HAMBRECHT + CO,WR HAMBRECHT + CO
501,YAMEX,YAMEX
502,ZURCHER KANTONALBANK,ZURCHER KANTONALBANK


In [72]:
import openai
import time

# Function to get an integer response from OpenAI API
def get_integer_from_openai(prompt):
    openai.api_key = os.getenv("OPENAI_KEY")
    
    max_retries = 10
    retries = 0
    while retries < max_retries:
        try:
            # Call the OpenAI API
            response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": " "},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
            )
            # Extract the text response from the API
            response_text = int(response.choices[0].message.content)
            break
        except Exception as e:
            retries += 1
            print('maxing out retries for the following error: ' + str(e))
            time.sleep(1)

    return response_text

In [83]:
import tqdm

gpt_match, gpt_match_for, gpt_match_against = 0, 0, 0
more_than_one_match, reverse_more_than_one_match = 0, 0
exact_match, reverse_match = 0, 0
unable_to_resolve = 0

res_dict = []
# we do first search: exact search by name and broker
for index, row in tqdm.tqdm(people_details.iterrows()):

    last_mame_option_list, reverse_last_name_option_list = [], []
    initial_option_list, reverse_initial_option_list = [], []
    # construct the name
    # 1. option
    try:
        last_mame_option_list.append(row['lastname'].upper())
        if len(row['lastname'].upper()) > 8:
            last_mame_option_list.append(row['lastname'].upper()[:8]) # as IBES has only 8 letters for lastname, cut the capital iq indicated last name
        initial_option_list.append(row['firstname'].upper()[0])

        reverse_last_name_option_list.append(row['firstname'].upper())
        if len(row['firstname'].upper()) > 8:
            reverse_last_name_option_list.append(row['firstname'].upper()[:8]) # as IBES has only 8 letters for lastname, cut the capital iq indicated last name

        reverse_initial_option_list.append(row['lastname'].upper()[0])
 
    except:
        pass
    # print('1st option: ' + initial_1option + ' ' + last_name_1option)
    
    # 2. option
    try:
        last_mame_option_list.append(row['transcriptpersonname'].upper().split(' ')[-1])
        if len(row['transcriptpersonname'].upper().split(' ')[-1]) > 8:
            last_mame_option_list.append(row['transcriptpersonname'].upper().split(' ')[-1][:8])
        initial_option_list.append(row['transcriptpersonname'].upper().split(' ')[0][0])

        reverse_last_name_option_list.append(row['transcriptpersonname'].upper().split(' ')[0])
        if len(row['transcriptpersonname'].upper().split(' ')[0]) > 8:
            reverse_last_name_option_list.append(row['transcriptpersonname'].upper().split(' ')[0][:8]) # as IBES has only 8 letters for lastname, cut the capital iq indicated last name

        reverse_initial_option_list.append(row['transcriptpersonname'].upper().split(' ')[-1][0])        
    except:
        pass
    # print('2nd option: ' + initial_2option + ' ' + last_name_2option)

    # deal with this two name options:
    if len(last_mame_option_list) == 0 or len(initial_option_list) == 0:
        unable_to_resolve += 1
        continue


    # construct the broker
    brokerid = row['estimatebrokerid']
    

    # exact match: exactly match the broker, analyst last name and analyst first name initials, extract the recpmmendation that are within same 10 years of the transcript
    IBES_matched = recommend_augmented.query("LastName in @last_mame_option_list").query("FirstName in @initial_option_list").query("choice == @brokerid").query("year <= 10 + @row['max_year']").query("year + 10 >= @row['min_year']")
    IBES_matched.drop_duplicates(subset=['EMASKCD'], inplace=True)

    # print(IBES_matched_by_name)
    if len(IBES_matched['EMASKCD'])>1:
        more_than_one_match += 1
        # regulate output
        for i in range(len(IBES_matched)):
            res_dict.append({'et_personid': row['transcriptpersonid'], 
                        'et_personname': row['transcriptpersonname'],
                        'et_companyname': row['companyname'], 
                        # 'et_personname2': row['firstname'] + ' ' + row['lastname'],
                        'et_broker': row['brokername'],
                        'et_brokerid': row['estimatebrokerid'],
                        'et_proid': row['proid'],
                        'et_earliest_year': row['min_year'],
                        'et_latest_year': row['max_year'],

                        'ibes_personid': IBES_matched['EMASKCD'].values[i],
                        'ibes_personname': IBES_matched['ANALYST'].values[i],
                        'ibes_broker': IBES_matched['ESTIMID'].values[i],
                        'match' : 'exact_onetomore'
                        })

    elif len(IBES_matched['EMASKCD'])==1:
        exact_match += 1
        # regulate output
        res_dict.append({'et_personid': row['transcriptpersonid'], 
                         'et_personname': row['transcriptpersonname'], 
                         'et_companyname': row['companyname'],
                        #  'et_personname2': row['firstname'] + ' ' + row['lastname'],
                         'et_broker': row['brokername'],
                         'et_brokerid': row['estimatebrokerid'],
                         'et_proid': row['proid'],
                         'et_earliest_year': row['min_year'],
                         'et_latest_year': row['max_year'],

                         'ibes_personid': IBES_matched['EMASKCD'].values[0],
                         'ibes_personname': IBES_matched['ANALYST'].values[0],
                         'ibes_broker': IBES_matched['ESTIMID'].values[0],
                         'match' : 'exact_onetoone'
                         })
        # print(res_dict)

    elif len(IBES_matched) == 0:
        # second, reverse last name first name search, but otherwise still use exact match method
        IBES_matched = recommend_augmented.query("choice == @brokerid").query("LastName in @reverse_last_name_option_list").query("FirstName in @reverse_initial_option_list").query("year <= 10 + @row['max_year']").query("year + 10 >= @row['min_year']")
        IBES_matched.drop_duplicates(subset=['EMASKCD'], inplace=True)
        if len(IBES_matched['EMASKCD'])>1:
            reverse_more_than_one_match += 1
            # regulate output
            for i in range(len(IBES_matched)):
                res_dict.append({'et_personid': row['transcriptpersonid'], 
                            'et_personname': row['transcriptpersonname'],
                            'et_companyname': row['companyname'],
                            # 'et_personname2': row['firstname'] + ' ' + row['lastname'],
                            'et_broker': row['brokername'],
                            'et_brokerid': row['estimatebrokerid'],
                            'et_proid': row['proid'],
                            'et_earliest_year': row['min_year'],
                            'et_latest_year': row['max_year'],

                            'ibes_personid': IBES_matched['EMASKCD'].values[i],
                            'ibes_personname': IBES_matched['ANALYST'].values[i],
                            'ibes_broker': IBES_matched['ESTIMID'].values[i],
                            'match' : 'reverse_onetomore'
                            })

        elif len(IBES_matched['EMASKCD'])==1:
            reverse_match += 1
            # regulate output
            res_dict.append({'et_personid': row['transcriptpersonid'], 
                            'et_personname': row['transcriptpersonname'], 
                            # 'et_personname2': row['firstname'] + ' ' + row['lastname'],
                            'et_broker': row['brokername'],
                            'et_brokerid': row['estimatebrokerid'],
                            'et_proid': row['proid'],
                            'et_earliest_year': row['min_year'],
                            'et_latest_year': row['max_year'],

                            'ibes_personid': IBES_matched['EMASKCD'].values[0],
                            'ibes_personname': IBES_matched['ANALYST'].values[0],
                            'ibes_broker': IBES_matched['ESTIMID'].values[0],
                            'match' : 'reverse_onetoone'
                            })
        else:
            # fuzzy match: fuzzy match the broker, exact match analyst last name and analyst first name initials
            IBES_matched = recommend_augmented.query("LastName in @last_mame_option_list").query("FirstName in @initial_option_list").query("year <= 10 + @row['max_year']").query("year + 10 >= @row['min_year']")
            IBES_matched.drop_duplicates(subset=['EMASKCD'], inplace=True)

            if len(IBES_matched) >= 1:
                gpt_match += 1
                for _, ibes_matched_row in IBES_matched.iterrows():
                    ibes_personname = ibes_matched_row['ANALYST']
                    # try to get broker name with its ESTIMID, and choice
                    ibes_brokername = list(broker_mapping.query("ESTIMID == @ibes_matched_row['ESTIMID']")['brokername'].values)
                    _brokernameeu = list(europe_match.query("ESTIMID == @ibes_matched_row['ESTIMID']")['brokername_eu'].values)
                    _brokernameold = list(old_match.query("ESTIMID == @ibes_matched_row['ESTIMID']")['brokername_old'].values)
                    
                    ibes_brokername.extend(_brokernameeu)
                    ibes_brokername.extend(_brokernameold)


                    _ciq_broker_name = []
                    for x in [row['brokername'], row['companyname']]:
                        if x is not None:
                            _ciq_broker_name.append(x)
                    # print(_ciq_broker_name)
                    
                    prompt = f"""
                    You are a researcher, you are expert in equity analyst. Please try to decide whether the profile provided by data provider CapitalIQ is the same profile provided by IBES, based on the following info:
                    1. the last name of the analyst from CapitalIQ is one of the following: {set(last_mame_option_list)} or {set(reverse_last_name_option_list)}. The last name and first name initial from IBES is {ibes_personname}.
                    2. the analyst from CapitalIQ works for broker that has name: {_ciq_broker_name}, the analyst from IBES works for broker that has the name of the one of the following: {ibes_brokername}; the IBES broker id is {set(IBES_matched['ESTIMID'].values)}. 
                    Note that sometimes even if the broker name are not the same, maybe they are linked: for example, the broker name in CapitalIQ is 'Morgan Stanley', the broker name in IBES is 'Dean Witter', they are actually the same broker.

                    It is a task of fuzzy match, you can not be 100% sure, but you should try your best to make the decision. Be cautious, the decision will be used for further analysis.

                    Are they representing the same equity analyst? Please answer the question by returning 1 if they are the same analyst or 0 if they are not the same analyst or you are not sure.
                    return only the integer and nothing else

                    """ 
                    # print(prompt)
                    result = get_integer_from_openai(prompt)

                    if result == 1:
                        # regulate output
                        gpt_match_for += 1
                        res_dict.append({'et_personid': row['transcriptpersonid'], 
                                        'et_personname': row['transcriptpersonname'], 
                                        'et_companyname': row['companyname'],
                                        # 'et_personname2': row['firstname'] + ' ' + row['lastname'],
                                        'et_broker': row['brokername'],
                                        'et_brokerid': row['estimatebrokerid'],
                                        'et_proid': row['proid'],
                                        'et_earliest_year': row['min_year'],
                                        'et_latest_year': row['max_year'],

                                        'ibes_personid': ibes_matched_row['EMASKCD'],
                                        'ibes_personname': ibes_matched_row['ANALYST'],
                                        'ibes_broker': ibes_matched_row['ESTIMID'],
                                        'match' : 'gpt_match'
                                        })
                    if result == 0:
                        gpt_match_against += 1

            else:
                unable_to_resolve += 1


print("total analysts: " + str(len(people_details)))
print("more than one match: " + str(more_than_one_match))
print("exact match: " + str(exact_match))
print("reverse more than one match: " + str(reverse_more_than_one_match))
print("reverse exact match: " + str(reverse_match))
print("gpt match: " + str(gpt_match))
print("unable to resolve: " + str(unable_to_resolve))

print("gpt match for: " + str(gpt_match_for) + " gpt match against: " + str(gpt_match_against))
print("checksum: " + str(more_than_one_match + exact_match + unable_to_resolve + reverse_more_than_one_match + reverse_match + gpt_match))

2994it [10:59,  4.54it/s]

total analysts: 2994
more than one match: 22
exact match: 2159
reverse more than one match: 0
reverse exact match: 0
gpt match: 437
unable to resolve: 376
gpt match for: 118 gpt match against: 832
checksum: 2994


In [84]:
res = pd.DataFrame(res_dict)
res.to_csv('../data/analyst_match_v2.csv', index=False)
res

res.query("match == 'gpt_match'")

,et_personid,et_personname,et_companyname,et_broker,et_brokerid,et_proid,et_earliest_year,et_latest_year,ibes_personid,ibes_personname,ibes_broker,match
0,4365,Christopher Glynn,Oppenheimer,None,NaN,NaN,2012,2012,1750.0,GLYNN C,OPPEN,gpt_match
1,4365,Christopher Glynn,Oppenheimer,None,NaN,NaN,2012,2012,836.0,GLYNN C,FAHN,gpt_match
2,4745,Kurt Hallead,RBC Capital Markets,None,NaN,NaN,2013,2013,659.0,HALLEAD K,RBCDOMIN,gpt_match
3,4896,Paul Ridzon,KeyBanc,None,NaN,NaN,2012,2012,1485.0,RIDZON P,MCDONALD,gpt_match
4,8196,Michael Lapides,Goldman Sachs,None,NaN,NaN,2012,2012,1020.0,LAPIDES M,GOLDMAN,gpt_match
...,...,...,...,...,...,...,...,...,...,...,...,...
2182,405463,Richard Greenfield,None,None,NaN,642175991.0,2020,2020,39605.0,GREENFIELD R,BTIG,gpt_match
2183,405716,Walter Piecyk,None,None,NaN,642863331.0,2020,2022,39605.0,PIECYK W,BTIG,gpt_match
2232,444375,Christopher Danely,None,None,NaN,660613802.0,2022,2022,1243.0,DANELY C,JPMORGAN,gpt_match
2246,456368,Andrew Kaplowitz,None,None,NaN,680961563.0,2021,2021,10902.0,KAPLOWITZ A,FRCLAYSC,gpt_match


for about 1000 transcripts, i extracted about 2994 distinctinct analysts who posted questions that not only have name information but also have employer information. (note that it will be considered as different people if the same person work for 2 different firms in cpaitalIQ. but in IBES, EMASKCD maintains the same in pirinciple across differnt work experience.)

Out of this 2994 distinct analysts in CapitalIQ, For each analyst, I built a [posisble analysts last name, possible analyst first name initials, possible employer, possible working years span] quadruple, and use this quadruple to match IBES analysts. 

Out of 2994 analyts, 
- 2159 has exactly 1 match (i.e. there is only one that satisfy a matching of the quadruple criterion.)
- 22 has more than 1 match (i.e. there are more than one analysts in IBES that mapped to CapitalIQ analyst) 
- 0 are matched by reverse last name and first name (i guess it is becasue of the US equity analysts tend not to have a lot of difficult mainland Chinese, jp, or korean names, e.g. she might be a jessica Lau, then it's not too hard to match...)
- 437 analysts are attempted to be matched by using GPT (fuzzy match, where the last name have to be the same, (or the same up to 8 letters), but the broker mapped do not need to be the same)
1. out of the gpt attempted matches, 118 are matched for, 832 are against.
- 376 analysts could not be matched (namely 376 analysts that asked a question in Q&A in CapitalIQ could not be matched to an analyst making stock recommendations in IBES)


A table is provided as attached.